# Personalized Learning Demo - Quickstart

A full-stack A2UI sample demonstrating personalized educational content generation.

**Contributed by Google Public Sector's Rapid Innovation Team.**

---

## What This Demo Shows

This demo showcases how A2UI enables AI agents to generate rich, interactive learning materials tailored to individual learners:

- **Flashcards** - Generated from OpenStax textbook content
- **Audio** - Personalized podcasts (via NotebookLM)
- **Video** - Educational explainers
- **Quizzes** - Interactive assessment

### The Personalization Pipeline

At Google Public Sector, we're developing approaches that combine LLMs, knowledge graphs, and learner performance data to produce personalized content across courses—and across a person's academic and professional life.

For this demo, that personalization is represented by context files in `learner_context/` describing a fictional learner (Maria) and her learning needs.

---

## Prerequisites

- Google Cloud project with billing enabled
- `gcloud` CLI installed
- Node.js 18+
- Python 3.11+

## Step 1: Set Your Project ID

Replace with your GCP project ID:

In [ ]:
PROJECT_ID = "your-project-id"  # <-- CHANGE THIS
LOCATION = "us-central1"

## Step 2: Authenticate & Enable APIs

In [ ]:
# Authenticate with Google Cloud
!gcloud auth login
!gcloud config set project {PROJECT_ID}
!gcloud auth application-default login

In [ ]:
# Enable required APIs
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable storage.googleapis.com

In [ ]:
# Create staging bucket for Agent Engine (if it doesn't exist)
!gsutil mb -l {LOCATION} gs://{PROJECT_ID}_cloudbuild 2>/dev/null || echo "Bucket already exists"

## Step 3: Deploy the A2UI Agent

The agent generates personalized learning content and runs on Vertex AI Agent Engine.

In [ ]:
# Install Python dependencies
!pip install -q google-adk google-genai vertexai python-dotenv litellm

In [ ]:
# Deploy the agent (takes 2-5 minutes)
!cd agent && python deploy.py --project {PROJECT_ID} --location {LOCATION}

**Save the Resource ID from the output above!** You'll need it in the next step.

In [ ]:
# Get your project NUMBER (different from project ID)
import subprocess
result = subprocess.run(["gcloud", "projects", "describe", PROJECT_ID, "--format=value(projectNumber)"], 
                       capture_output=True, text=True)
PROJECT_NUMBER = result.stdout.strip()
print(f"Project Number: {PROJECT_NUMBER}")

## Step 4: Configure Environment

Fill in the Resource ID from the deployment output:

In [ ]:
AGENT_RESOURCE_ID = ""  # <-- PASTE YOUR RESOURCE ID HERE (from Step 3 output)

In [ ]:
# Create .env file
env_content = f"""# Generated by Quickstart.ipynb
GOOGLE_CLOUD_PROJECT={PROJECT_ID}
AGENT_ENGINE_PROJECT_NUMBER={PROJECT_NUMBER}
AGENT_ENGINE_RESOURCE_ID={AGENT_RESOURCE_ID}
"""

with open(".env", "w") as f:
    f.write(env_content)

print("Created .env file:")
print(env_content)

## Step 5: Install Frontend Dependencies

In [ ]:
# Build the A2UI renderer first
!cd ../../renderers/lit && npm install && npm run build

In [ ]:
# Install demo dependencies
!npm install

## Step 6: Run the Demo

Run this in your terminal (not in the notebook):

```bash
npm run dev
```

Then open http://localhost:5174

### Try These Prompts

| Prompt | What Happens |
|--------|-------------|
| "Help me understand ATP" | Generates flashcards from OpenStax |
| "Quiz me on bond energy" | Interactive quiz cards |
| "Play the podcast" | Audio player (requires Step 7) |
| "Show me a video" | Video player (requires Step 7) |

---

## Step 7 (Optional): Generate Audio/Video with NotebookLM

The demo can show audio and video players, but you need to generate the content first.

### Generate a Podcast

1. Go to [notebooklm.google.com](https://notebooklm.google.com/)
2. Create a new notebook
3. Upload all files from `learner_context/`
4. Click **Audio Overview** to generate a podcast
5. Download and save as `public/assets/podcast.m4a`

### Generate a Video

Use NotebookLM's briefing document + screen recording, or create slides with voiceover.
Save as `public/assets/demo.mp4`

### Programmatic Alternatives

- **Podcast API** - Google's podcast generation API can create audio programmatically
- **Gemini Image Generation** - Create educational diagrams via Imagen

---

## Content Attribution

### OpenStax

Educational content is sourced from [OpenStax](https://openstax.org/), licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/).

Specifically: [Biology for AP® Courses](https://openstax.org/details/books/biology-ap-courses) - OpenStax, Rice University

---

## Security Notice

> **Warning:** When building production applications, treat any agent outside your control as potentially untrusted. This demo connects to Agent Engine within your own GCP project. Always review agent code before deploying.